# ⬅️ Backpropagation - Comment un Réseau Apprend

Bienvenue dans le notebook le plus important : la **backpropagation** !

## 🎯 Objectifs

1. **Comprendre l'intuition** derrière la backpropagation
2. **Voir les mathématiques** (mais simplement !)
3. **Implémenter** l'algorithme étape par étape
4. **Visualiser** comment le réseau s'améliore

---

## 💡 L'Intuition en 3 Questions

### 1. Le réseau fait une erreur. Pourquoi ?
**Réponse** : Parce que les poids ne sont pas bons.

### 2. Comment savoir quels poids sont responsables ?
**Réponse** : En propageant l'erreur **en arrière** dans le réseau.

### 3. Comment corriger les poids ?
**Réponse** : En les ajustant dans la direction qui **réduit l'erreur**.

C'est ça, la backpropagation ! 🎓

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

sys.path.append(str(Path.cwd().parent))

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

np.random.seed(42)

print("✅ Prêt à apprendre la backpropagation !")

## 1️⃣ Analogie : Apprendre à Tirer au But

Imagine que tu apprends à tirer au football :

1. 🎯 **Tu tires** → Le ballon va à gauche du but
2. 🤔 **Tu analyses** → "J'ai tiré trop à gauche"
3. 🔧 **Tu ajustes** → "La prochaine fois, je vise plus à droite"
4. 🔄 **Tu recommences** → Tu t'améliores !

### La backpropagation fait EXACTEMENT pareil :

1. 🎯 **Forward pass** → Le réseau fait une prédiction
2. 📊 **Calcul de l'erreur** → "Voilà à quel point je me suis trompé"
3. ⬅️ **Backward pass** → "Ces poids sont responsables de l'erreur"
4. 🔧 **Mise à jour** → "Je les ajuste pour faire mieux"
5. 🔄 **Répète** → Le réseau s'améliore !

## 2️⃣ La Fonction de Coût (Loss Function)

Avant de corriger, il faut **mesurer l'erreur** !

### Cross-Entropy Loss

Pour la classification, on utilise la **cross-entropy** :

$$
L = -\sum_{i=0}^{9} y_i \log(\hat{y}_i)
$$

Où :
- $y$ = vrai label (one-hot encoded)
- $\hat{y}$ = prédiction (probabilités)

### Pourquoi cette formule ?

- Si prédiction **correcte** (ex: 95% sur la bonne classe) → Loss **petite** ✅
- Si prédiction **incorrecte** (ex: 5% sur la bonne classe) → Loss **grande** ❌

In [ ]:
def cross_entropy_loss(y_true, y_pred):
    """
    Calcule la cross-entropy loss
    
    Args:
        y_true: vrais labels (one-hot encoded) - shape: (n_samples, n_classes)
        y_pred: prédictions (probabilités) - shape: (n_samples, n_classes)
    
    Returns:
        loss: erreur moyenne
    """
    n_samples = y_true.shape[0]
    
    # Éviter log(0) en ajoutant une petite valeur
    epsilon = 1e-7
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    
    # Cross-entropy
    loss = -np.sum(y_true * np.log(y_pred)) / n_samples
    
    return loss

def one_hot_encode(y, n_classes=10):
    """
    Transforme les labels en one-hot encoding
    
    Exemple: 3 → [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    """
    one_hot = np.zeros((y.shape[0], n_classes))
    one_hot[np.arange(y.shape[0]), y] = 1
    return one_hot

# Test
print("🧪 Test de la fonction de coût:\n")

# Cas 1: Bonne prédiction
y_true_1 = one_hot_encode(np.array([3]))
y_pred_1 = np.array([[0.01, 0.01, 0.01, 0.95, 0.01, 0.01, 0.00, 0.00, 0.00, 0.00]])
loss_1 = cross_entropy_loss(y_true_1, y_pred_1)
print(f"✅ Bonne prédiction (95% sur classe 3):")
print(f"   Loss = {loss_1:.4f} (petite!)\n")

# Cas 2: Mauvaise prédiction
y_true_2 = one_hot_encode(np.array([3]))
y_pred_2 = np.array([[0.05, 0.05, 0.05, 0.10, 0.25, 0.15, 0.10, 0.10, 0.10, 0.05]])
loss_2 = cross_entropy_loss(y_true_2, y_pred_2)
print(f"❌ Mauvaise prédiction (10% sur classe 3):")
print(f"   Loss = {loss_2:.4f} (grande!)\n")

print(f"💡 La loss est {loss_2/loss_1:.1f}x plus grande quand la prédiction est mauvaise!")

### 📊 Visualisons la Loss

In [ ]:
def visualize_loss_function():
    """
    Montre comment varie la loss selon la confiance de la prédiction
    """
    # Probabilités prédites pour la bonne classe
    probabilities = np.linspace(0.01, 0.99, 100)
    losses = -np.log(probabilities)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    
    # Graphique 1: Loss vs Probabilité
    ax1.plot(probabilities, losses, linewidth=3, color='#e74c3c')
    ax1.fill_between(probabilities, 0, losses, alpha=0.3, color='#e74c3c')
    ax1.set_xlabel('Probabilité prédite pour la VRAIE classe', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Loss (Cross-Entropy)', fontsize=13, fontweight='bold')
    ax1.set_title('📈 Comment la Loss Pénalise les Mauvaises Prédictions', 
                 fontsize=15, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Annotations
    ax1.annotate('Très mauvaise\nprédiction\n(Loss élevée)', 
                xy=(0.1, -np.log(0.1)), xytext=(0.3, 3),
                fontsize=11, ha='center',
                bbox=dict(boxstyle='round', facecolor='red', alpha=0.7),
                arrowprops=dict(arrowstyle='->', lw=2, color='red'))
    
    ax1.annotate('Bonne\nprédiction\n(Loss faible)', 
                xy=(0.9, -np.log(0.9)), xytext=(0.7, 1),
                fontsize=11, ha='center',
                bbox=dict(boxstyle='round', facecolor='green', alpha=0.7),
                arrowprops=dict(arrowstyle='->', lw=2, color='green'))
    
    # Graphique 2: Exemples concrets
    scenarios = ['Prédiction\nparfaite\n(99%)', 'Bonne\nprédiction\n(80%)', 
                'Prédiction\nmoyenne\n(50%)', 'Mauvaise\nprédiction\n(20%)', 
                'Très mauvaise\nprédiction\n(5%)']
    probs = [0.99, 0.80, 0.50, 0.20, 0.05]
    colors_bar = ['darkgreen', 'lightgreen', 'yellow', 'orange', 'red']
    losses_scenarios = [-np.log(p) for p in probs]
    
    bars = ax2.bar(range(5), losses_scenarios, color=colors_bar, 
                   edgecolor='black', linewidth=2, alpha=0.8)
    ax2.set_xlabel('Scénario', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Loss', fontsize=13, fontweight='bold')
    ax2.set_title('🎯 Loss pour Différents Scénarios', fontsize=15, fontweight='bold')
    ax2.set_xticks(range(5))
    ax2.set_xticklabels(scenarios, fontsize=10)
    ax2.grid(axis='y', alpha=0.3)
    
    # Valeurs sur les barres
    for i, (bar, loss, prob) in enumerate(zip(bars, losses_scenarios, probs)):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{loss:.3f}',
                ha='center', va='bottom', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 À retenir:")
    print("   • Plus la prédiction est confiante et CORRECTE → Loss faible")
    print("   • Plus la prédiction est mauvaise → Loss élevée")
    print("   • La loss 'explose' quand la prédiction est très mauvaise")
    print("   • L'objectif de l'entraînement: MINIMISER cette loss !")

visualize_loss_function()

## 3️⃣ Le Gradient : La Direction de la Descente

### 🎿 Analogie : Descendre une Montagne

Imagine que tu es sur une montagne dans le brouillard :
- ❓ Tu ne vois pas où est le bas
- 🔍 Mais tu peux sentir la pente sous tes pieds
- 🚶 Tu marches dans la direction de la **plus grande pente**
- 🎯 Petit à petit, tu arrives en bas !

### C'est exactement la descente de gradient !

- 🏔️ **Montagne** = Surface de la loss (fonction à minimiser)
- 📍 **Ta position** = Valeurs actuelles des poids
- 📐 **La pente** = Le **gradient** (dérivée de la loss)
- 🚶 **Marcher** = Ajuster les poids
- 🎯 **En bas** = Loss minimale = bon réseau !

### Formule de mise à jour :

$$
W_{\text{nouveau}} = W_{\text{ancien}} - \alpha \cdot \frac{\partial L}{\partial W}
$$

Où :
- $\alpha$ = **learning rate** (taille du pas)
- $\frac{\partial L}{\partial W}$ = **gradient** (direction de la pente)

In [ ]:
def visualize_gradient_descent():
    """
    Visualise la descente de gradient sur une fonction simple
    """
    # Fonction simple: f(x) = x^2
    x = np.linspace(-3, 3, 100)
    y = x ** 2
    
    # Simulation de descente de gradient
    learning_rates = [0.1, 0.3, 0.5]
    colors = ['#3498db', '#2ecc71', '#e74c3c']
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    fig.suptitle('🎿 Descente de Gradient avec Différents Learning Rates', 
                fontsize=18, fontweight='bold')
    
    for ax, lr, color in zip(axes, learning_rates, colors):
        # Tracer la fonction
        ax.plot(x, y, 'k-', linewidth=2, alpha=0.3, label='Loss = x²')
        
        # Descente de gradient
        x_current = 2.5  # Point de départ
        trajectory_x = [x_current]
        trajectory_y = [x_current ** 2]
        
        for i in range(15):
            gradient = 2 * x_current  # Dérivée de x^2 = 2x
            x_current = x_current - lr * gradient
            trajectory_x.append(x_current)
            trajectory_y.append(x_current ** 2)
        
        # Tracer la trajectoire
        ax.plot(trajectory_x, trajectory_y, 'o-', color=color, 
               linewidth=2, markersize=8, label=f'Trajectoire (α={lr})')
        
        # Point de départ et d'arrivée
        ax.plot(trajectory_x[0], trajectory_y[0], 'ro', markersize=15, 
               label='Départ', zorder=5)
        ax.plot(trajectory_x[-1], trajectory_y[-1], 'go', markersize=15, 
               label='Arrivée', zorder=5)
        
        ax.set_xlabel('Poids (W)', fontsize=12, fontweight='bold')
        ax.set_ylabel('Loss', fontsize=12, fontweight='bold')
        ax.set_title(f'Learning Rate α = {lr}', fontsize=14, fontweight='bold')
        ax.legend(fontsize=10)
        ax.grid(True, alpha=0.3)
        ax.set_ylim(-0.5, 7)
    
    plt.tight_layout()
    plt.show()
    
    print("\n🎓 Observations:")
    print("\n   📉 Learning Rate α = 0.1 (petit):")
    print("      • Progression lente mais sûre")
    print("      • Converge doucement vers le minimum")
    print("      ✅ Sûr mais peut être lent\n")
    
    print("   📉 Learning Rate α = 0.3 (moyen):")
    print("      • Progression rapide")
    print("      • Converge efficacement")
    print("      ✅ Bon équilibre!\n")
    
    print("   📉 Learning Rate α = 0.5 (grand):")
    print("      • Progression très rapide")
    print("      • Peut osciller autour du minimum")
    print("      ⚠️ Risque d'instabilité\n")
    
    print("💡 Conclusion: Le learning rate doit être bien choisi!")

visualize_gradient_descent()

## 4️⃣ Backpropagation : Les Mathématiques

Maintenant, voyons comment calculer ces gradients !

### Rappel de notre réseau :

```
Forward:
  X → [W1, b1, ReLU] → A1 → [W2, b2, Softmax] → A2 → Loss

Backward (on inverse!):
  ∂Loss/∂W1 ← [backprop] ← ∂Loss/∂A1 ← [backprop] ← ∂Loss/∂A2 ← Loss
```

### Les gradients à calculer :

#### Couche de sortie (Layer 2) :
$$\frac{\partial L}{\partial Z_2} = A_2 - Y \quad \text{(pour Softmax + Cross-Entropy)}$$

$$\frac{\partial L}{\partial W_2} = A_1^T \cdot \frac{\partial L}{\partial Z_2}$$

$$\frac{\partial L}{\partial b_2} = \sum \frac{\partial L}{\partial Z_2}$$

#### Couche cachée (Layer 1) :
$$\frac{\partial L}{\partial A_1} = \frac{\partial L}{\partial Z_2} \cdot W_2^T$$

$$\frac{\partial L}{\partial Z_1} = \frac{\partial L}{\partial A_1} \odot \text{ReLU}'(Z_1) \quad \text{(où ReLU' = 1 si Z>0, 0 sinon)}$$

$$\frac{\partial L}{\partial W_1} = X^T \cdot \frac{\partial L}{\partial Z_1}$$

$$\frac{\partial L}{\partial b_1} = \sum \frac{\partial L}{\partial Z_1}$$

### 💡 Pas de panique !
Ces formules peuvent sembler compliquées, mais ce sont juste des multiplications matricielles ! 
Regardons le code, c'est plus clair :

In [ ]:
# D'abord, récupérons les fonctions du notebook précédent
def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))
    return exp_Z / np.sum(exp_Z, axis=1, keepdims=True)

def forward_propagation(X, parameters):
    W1, b1 = parameters['W1'], parameters['b1']
    W2, b2 = parameters['W2'], parameters['b2']
    
    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)
    Z2 = np.dot(A1, W2) + b2
    A2 = softmax(Z2)
    
    cache = {'Z1': Z1, 'A1': A1, 'Z2': Z2, 'A2': A2}
    return A2, cache

def backward_propagation(X, Y, parameters, cache, verbose=False):
    """
    Calcule les gradients par backpropagation
    
    Args:
        X: données d'entrée (n_samples, 784)
        Y: vrais labels (n_samples, 10) - one-hot encoded
        parameters: poids et biais actuels
        cache: valeurs de la forward propagation
        verbose: afficher les détails
    
    Returns:
        gradients: dictionnaire avec dW1, db1, dW2, db2
    """
    n_samples = X.shape[0]
    
    # Récupérer les valeurs du cache
    A1 = cache['A1']
    A2 = cache['A2']
    Z1 = cache['Z1']
    
    W2 = parameters['W2']
    
    if verbose:
        print("\n" + "="*70)
        print("⬅️ BACKPROPAGATION - ÉTAPE PAR ÉTAPE")
        print("="*70)
    
    # ===== COUCHE 2 (Output) =====
    # Gradient de la loss par rapport à Z2
    # Pour Softmax + Cross-Entropy, c'est super simple!
    dZ2 = A2 - Y
    
    if verbose:
        print(f"\n🔸 Layer 2 - Gradient par rapport à Z2:")
        print(f"   dZ2 = A2 - Y (simplifié pour Softmax+CrossEntropy)")
        print(f"   dZ2.shape = {dZ2.shape}")
        print(f"   dZ2 range: [{dZ2.min():.3f}, {dZ2.max():.3f}]")
    
    # Gradient par rapport à W2 et b2
    dW2 = np.dot(A1.T, dZ2) / n_samples
    db2 = np.sum(dZ2, axis=0, keepdims=True) / n_samples
    
    if verbose:
        print(f"\n🔸 Layer 2 - Gradients des paramètres:")
        print(f"   dW2 = A1.T · dZ2 / n_samples")
        print(f"   dW2.shape = {dW2.shape}")
        print(f"   db2.shape = {db2.shape}")
    
    # ===== COUCHE 1 (Hidden) =====
    # Propager le gradient vers A1
    dA1 = np.dot(dZ2, W2.T)
    
    if verbose:
        print(f"\n🔸 Layer 1 - Propagation du gradient:")
        print(f"   dA1 = dZ2 · W2.T")
        print(f"   dA1.shape = {dA1.shape}")
    
    # Gradient à travers ReLU
    # Dérivée de ReLU: 1 si Z1 > 0, 0 sinon
    dZ1 = dA1 * (Z1 > 0)
    
    if verbose:
        print(f"\n🔸 Layer 1 - Gradient à travers ReLU:")
        print(f"   dZ1 = dA1 ⊙ ReLU'(Z1)")
        print(f"   dZ1.shape = {dZ1.shape}")
        active = np.sum(Z1 > 0)
        print(f"   Gradients non-nuls: {active} / {Z1.size} ({active/Z1.size*100:.1f}%)")
    
    # Gradient par rapport à W1 et b1
    dW1 = np.dot(X.T, dZ1) / n_samples
    db1 = np.sum(dZ1, axis=0, keepdims=True) / n_samples
    
    if verbose:
        print(f"\n🔸 Layer 1 - Gradients des paramètres:")
        print(f"   dW1 = X.T · dZ1 / n_samples")
        print(f"   dW1.shape = {dW1.shape}")
        print(f"   db1.shape = {db1.shape}")
        print("\n" + "="*70)
        print("✅ BACKPROPAGATION COMPLETE")
        print("="*70)
    
    gradients = {
        'dW1': dW1, 'db1': db1,
        'dW2': dW2, 'db2': db2
    }
    
    return gradients

print("✅ Fonction backward_propagation créée !")

## 5️⃣ Test de la Backpropagation

In [ ]:
# Créer un mini réseau
def initialize_parameters(layer_dims):
    np.random.seed(42)
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        parameters[f'W{l}'] = np.random.randn(layer_dims[l-1], layer_dims[l]) * np.sqrt(2 / layer_dims[l-1])
        parameters[f'b{l}'] = np.zeros((1, layer_dims[l]))
    return parameters

# Charger quelques données
from src.utils import load_mnist_data
X_train, y_train, X_test, y_test = load_mnist_data()

# Prendre un mini-batch
batch_size = 5
X_batch = X_train[:batch_size]
Y_batch = one_hot_encode(y_train[:batch_size])

# Initialiser le réseau
layer_dims = [784, 128, 10]
params = initialize_parameters(layer_dims)

# Forward pass
print("\n🚀 FORWARD PASS")
predictions, cache = forward_propagation(X_batch, params)
loss = cross_entropy_loss(Y_batch, predictions)
print(f"\n📊 Loss avant backprop: {loss:.4f}")

# Backward pass
print("\n" + "="*70)
gradients = backward_propagation(X_batch, Y_batch, params, cache, verbose=True)

# Afficher les gradients
print("\n\n📊 MAGNITUDES DES GRADIENTS:")
print("="*70)
for key, grad in gradients.items():
    print(f"\n{key}:")
    print(f"   Shape: {grad.shape}")
    print(f"   Mean: {np.mean(np.abs(grad)):.6f}")
    print(f"   Max: {np.max(np.abs(grad)):.6f}")
    print(f"   Min: {np.min(np.abs(grad)):.6f}")

## 6️⃣ Mise à Jour des Poids

Maintenant qu'on a les gradients, on peut mettre à jour les poids !

In [ ]:
def update_parameters(parameters, gradients, learning_rate):
    """
    Met à jour les paramètres avec la descente de gradient
    
    Formule: W = W - α * dW
    """
    parameters_updated = {}
    
    for key in parameters.keys():
        grad_key = 'd' + key
        parameters_updated[key] = parameters[key] - learning_rate * gradients[grad_key]
    
    return parameters_updated

# Test
learning_rate = 0.01
params_before = {k: v.copy() for k, v in params.items()}
params_updated = update_parameters(params, gradients, learning_rate)

print("\n🔧 MISE À JOUR DES POIDS")
print("="*70)
print(f"\nLearning rate: {learning_rate}")

# Calculer les changements
for key in params_before.keys():
    change = params_updated[key] - params_before[key]
    print(f"\n{key}:")
    print(f"   Changement moyen: {np.mean(np.abs(change)):.6f}")
    print(f"   Changement max: {np.max(np.abs(change)):.6f}")

# Vérifier si la loss a diminué
predictions_after, _ = forward_propagation(X_batch, params_updated)
loss_after = cross_entropy_loss(Y_batch, predictions_after)

print("\n" + "="*70)
print("📊 RÉSULTAT")
print("="*70)
print(f"\nLoss avant:  {loss:.6f}")
print(f"Loss après:  {loss_after:.6f}")
print(f"Différence:  {loss - loss_after:.6f}")

if loss_after < loss:
    print("\n✅ SUCCESS! La loss a diminué! Le réseau s'améliore!")
else:
    print("\n⚠️ La loss a augmenté (peut arriver avec un batch trop petit)")

## 7️⃣ Cycle Complet d'Entraînement

Mettons tout ensemble pour voir l'entraînement en action !

In [ ]:
def train_one_epoch(X, Y, parameters, learning_rate, batch_size=32):
    """
    Entraîne le réseau sur une époque complète
    """
    n_samples = X.shape[0]
    n_batches = n_samples // batch_size
    
    total_loss = 0
    
    for i in range(n_batches):
        # Mini-batch
        start = i * batch_size
        end = start + batch_size
        X_batch = X[start:end]
        Y_batch = Y[start:end]
        
        # Forward
        predictions, cache = forward_propagation(X_batch, parameters)
        
        # Loss
        loss = cross_entropy_loss(Y_batch, predictions)
        total_loss += loss
        
        # Backward
        gradients = backward_propagation(X_batch, Y_batch, parameters, cache)
        
        # Update
        parameters = update_parameters(parameters, gradients, learning_rate)
    
    avg_loss = total_loss / n_batches
    return parameters, avg_loss

def compute_accuracy(X, y, parameters):
    """
    Calcule l'accuracy du réseau
    """
    predictions, _ = forward_propagation(X, parameters)
    predicted_labels = np.argmax(predictions, axis=1)
    accuracy = np.mean(predicted_labels == y)
    return accuracy

# Entraîner sur quelques époques
print("\n🎓 ENTRAÎNEMENT DU RÉSEAU")
print("="*70)

# Paramètres
n_epochs = 5
learning_rate = 0.1
batch_size = 128

# Utiliser un sous-ensemble pour aller plus vite
n_train = 10000
X_train_subset = X_train[:n_train]
y_train_subset = y_train[:n_train]
Y_train_one_hot = one_hot_encode(y_train_subset)

# Réinitialiser le réseau
params = initialize_parameters([784, 128, 10])

# Stocker l'historique
history = {'loss': [], 'train_acc': [], 'test_acc': []}

print(f"\nConfiguration:")
print(f"  • Époques: {n_epochs}")
print(f"  • Learning rate: {learning_rate}")
print(f"  • Batch size: {batch_size}")
print(f"  • Exemples d'entraînement: {n_train:,}")
print(f"\nDébut de l'entraînement...\n")

for epoch in range(n_epochs):
    # Entraîner
    params, avg_loss = train_one_epoch(X_train_subset, Y_train_one_hot, 
                                       params, learning_rate, batch_size)
    
    # Calculer l'accuracy
    train_acc = compute_accuracy(X_train_subset, y_train_subset, params)
    test_acc = compute_accuracy(X_test[:1000], y_test[:1000], params)
    
    # Sauvegarder
    history['loss'].append(avg_loss)
    history['train_acc'].append(train_acc)
    history['test_acc'].append(test_acc)
    
    # Afficher
    print(f"Époque {epoch+1}/{n_epochs} - "
          f"Loss: {avg_loss:.4f} - "
          f"Train Acc: {train_acc:.2%} - "
          f"Test Acc: {test_acc:.2%}")

print("\n✅ Entraînement terminé!")

### 📈 Visualisons l'Apprentissage

In [ ]:
def plot_training_history(history):
    """
    Visualise l'évolution de la loss et de l'accuracy
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('📈 Évolution de l\'Entraînement', fontsize=18, fontweight='bold')
    
    epochs = range(1, len(history['loss']) + 1)
    
    # Loss
    ax1.plot(epochs, history['loss'], 'o-', linewidth=3, markersize=10, 
            color='#e74c3c', label='Loss')
    ax1.set_xlabel('Époque', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Loss', fontsize=13, fontweight='bold')
    ax1.set_title('📉 Évolution de la Loss', fontsize=15, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=12)
    
    # Accuracy
    ax2.plot(epochs, [acc*100 for acc in history['train_acc']], 
            'o-', linewidth=3, markersize=10, color='#2ecc71', label='Train Accuracy')
    ax2.plot(epochs, [acc*100 for acc in history['test_acc']], 
            's-', linewidth=3, markersize=10, color='#3498db', label='Test Accuracy')
    ax2.set_xlabel('Époque', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Accuracy (%)', fontsize=13, fontweight='bold')
    ax2.set_title('📈 Évolution de l\'Accuracy', fontsize=15, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.legend(fontsize=12)
    ax2.set_ylim(0, 100)
    
    plt.tight_layout()
    plt.show()
    
    # Statistiques finales
    print("\n" + "="*70)
    print("📊 STATISTIQUES FINALES")
    print("="*70)
    print(f"\n🎯 Loss:")
    print(f"   • Initiale: {history['loss'][0]:.4f}")
    print(f"   • Finale: {history['loss'][-1]:.4f}")
    print(f"   • Réduction: {(1 - history['loss'][-1]/history['loss'][0])*100:.1f}%")
    
    print(f"\n🎯 Train Accuracy:")
    print(f"   • Initiale: {history['train_acc'][0]:.2%}")
    print(f"   • Finale: {history['train_acc'][-1]:.2%}")
    print(f"   • Amélioration: +{(history['train_acc'][-1] - history['train_acc'][0])*100:.1f} points")
    
    print(f"\n🎯 Test Accuracy:")
    print(f"   • Initiale: {history['test_acc'][0]:.2%}")
    print(f"   • Finale: {history['test_acc'][-1]:.2%}")
    print(f"   • Amélioration: +{(history['test_acc'][-1] - history['test_acc'][0])*100:.1f} points")
    
    print("\n💡 Le réseau a appris ! 🎉")

plot_training_history(history)

## 🎯 Récapitulatif

Félicitations ! Tu as compris la **backpropagation** ! 🎉

### ✅ Ce que nous avons appris

1. **L'intuition** :
   - Le réseau fait une erreur
   - On propage cette erreur en arrière
   - On ajuste les poids pour réduire l'erreur

2. **La fonction de coût** :
   - Cross-entropy pour mesurer l'erreur
   - Plus la prédiction est mauvaise, plus la loss est grande

3. **Le gradient** :
   - Direction pour minimiser la loss
   - Descente de gradient pour optimiser
   - Learning rate contrôle la taille des pas

4. **L'implémentation** :
   - Calcul des gradients couche par couche
   - Mise à jour des poids: W = W - α·dW
   - Le réseau s'améliore à chaque itération !

### 🧠 Points clés

- 📊 **Backprop** = algorithme pour calculer les gradients efficacement
- 🎯 **Loss** diminue → le réseau s'améliore
- 📈 **Accuracy** augmente → le réseau apprend !
- 🔧 **Learning rate** : crucial pour la convergence

### 🚀 Prochaine Étape

Maintenant qu'on comprend tout le processus, construisons un réseau complet from scratch !

**➡️ Prochain notebook: `04_building_complete_network.ipynb`**

On va créer un système d'entraînement complet avec :
- Architecture modulaire
- Entraînement sur tout MNIST
- Visualisation des résultats
- Analyse des performances

---

**Bravo pour avoir maîtrisé la backpropagation ! C'est le concept le plus important en deep learning ! 🏆**